SQL Question 1: Rolling 3 Months Active Users per Product
Assume that Nagarro is an e-commerce company. Their product team wants to analyze user engagement on a rolling 3 months basis. Specifically, they want to calculate how many unique active users a product has on a rolling 3 months basis, sorted by date and product.

Use the schema and sample data from the reviews table provided below for this question.

reviews Example Input:
review_id	user_id	submit_date	product_id	stars
6171	123	2022-01-08	50001	4
7802	265	2022-02-10	69852	4
5293	362	2022-03-18	50001	3
6352	192	2022-04-26	69852	3
4517	981	2022-05-05	69852	2
We define an active user as a user that has submitted at least one review in the timeframe. Note: use the submit_date to calculate the rolling 3 month window.

Example Output:
end_of_period	product_id	active_users
2022-03-31	50001	2
2022-03-31	69852	1
2022-04-30	50001	1
2022-04-30	69852	2
2022-05-31	50001	0
2022-05-31	69852	2


In [11]:
from datetime import date
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Initialize Spark session
spark = SparkSession.builder.appName("CreateReviewDataFrame").getOrCreate()

In [18]:
# Define data for the DataFrame
data = [
    (6171, 123, date(2022, 1, 8), 50001, 4),
    (7802, 265, date(2022, 2, 10), 69852, 4),
    (5293, 362, date(2022, 3, 18), 50001, 3),
    (6352, 192, date(2022, 4, 26), 69852, 3),
    (4517, 981, date(2022, 5, 5), 69852, 2),
]

# Define schema
schema = "review_id int, user_id int, submit_date date, product_id int, stars int"

# Create DataFrame
df = spark.createDataFrame(data, schema)
df.createOrReplaceTempView("reviews")

# Show the DataFrame
df.show(F.)

+---------+-------+-----------+----------+-----+
|review_id|user_id|submit_date|product_id|stars|
+---------+-------+-----------+----------+-----+
|     6171|    123| 2022-01-08|     50001|    4|
|     7802|    265| 2022-02-10|     69852|    4|
|     5293|    362| 2022-03-18|     50001|    3|
|     6352|    192| 2022-04-26|     69852|    3|
|     4517|    981| 2022-05-05|     69852|    2|
+---------+-------+-----------+----------+-----+



In [57]:
query=""" 
Select end_of_period, product_id, count(Distinct user_id) AS active_users from
(Select *, last_day(submit_date) AS end_of_period, add_months(end_of_period, -3) AS start_of_period from reviews) sub
where submit_date >= start_of_period and submit_date <= end_of_period
GROUP BY end_of_period, product_id
"""

In [65]:
query2 = """SELECT 
 end_of_period,
 product_id,
 COUNT(DISTINCT user_id) AS active_users
FROM (
 SELECT
   last_day(submit_date) AS end_of_period,
   product_id,
   user_id
 FROM 
   reviews
 WHERE 
   submit_date BETWEEN 
   add_months(last_day(submit_date), -3) AND 
   last_day(submit_date)
) t
GROUP BY 
 end_of_period, 
 product_id
ORDER BY
 end_of_period ASC,
 product_id ASC;"""

In [66]:
spark.sql(query2).show()

+-------------+----------+------------+
|end_of_period|product_id|active_users|
+-------------+----------+------------+
|   2022-01-31|     50001|           1|
|   2022-02-28|     69852|           1|
|   2022-03-31|     50001|           1|
|   2022-04-30|     69852|           1|
|   2022-05-31|     69852|           1|
+-------------+----------+------------+

